In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Ingestion
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://medium.com/@tirthraj2004/apache-kafka-deep-dive-8f011c6483e1")
docs= loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://medium.com/@tirthraj2004/apache-kafka-deep-dive-8f011c6483e1', 'title': 'Apache Kafka Deep Dive. Apache Kafka Internals | by Tirthraj Mahajan | Jul, 2025 | Medium', 'description': 'It prepares the environment, handles optional flags (like -daemon), and then invokes the actual Java class that runs the Kafka server Sets default JVM heap size to 1 GB if not configured. You can…', 'language': 'en'}, page_content='Apache Kafka Deep Dive. Apache Kafka Internals | by Tirthraj Mahajan | Jul, 2025 | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inApache Kafka Deep DiveTirthraj Mahajan17 min read·Jul 26, 2025--ListenSharePress enter or click to view image in full sizeKafka ArchitectureProject StructureWe are going to refer the Apache Kafka Mirror on Github: https://github.com/apache/kafkaPress enter or click to view image in full sizeModules OverviewLet us take a look at the starting point of our deep dive. We’ll look at core module firs

In [ ]:
## splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
split_docs = splitter.split_documents(docs)

In [6]:
split_docs

[Document(metadata={'source': 'https://medium.com/@tirthraj2004/apache-kafka-deep-dive-8f011c6483e1', 'title': 'Apache Kafka Deep Dive. Apache Kafka Internals | by Tirthraj Mahajan | Jul, 2025 | Medium', 'description': 'It prepares the environment, handles optional flags (like -daemon), and then invokes the actual Java class that runs the Kafka server Sets default JVM heap size to 1 GB if not configured. You can…', 'language': 'en'}, page_content='Apache Kafka Deep Dive. Apache Kafka Internals | by Tirthraj Mahajan | Jul, 2025 | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inApache Kafka Deep DiveTirthraj Mahajan17 min read·Jul 26, 2025--ListenSharePress enter or click to view image in full sizeKafka ArchitectureProject StructureWe are going to refer the Apache Kafka Mirror on Github: https://github.com/apache/kafkaPress enter or click to view image in full sizeModules OverviewLet us take a look at the starting point of our deep dive. We’ll look at core module firs

In [7]:
## embeddings
from langchain_openai import OpenAIEmbeddings
embeddings= OpenAIEmbeddings()

In [8]:
from langchain_community.vectorstores import FAISS
vectordb = FAISS.from_documents(split_docs,embeddings)

In [9]:
vectordb

Document Chain And Retrieval Chain

In [10]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [12]:
#document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>

"""
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002081F664A60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002081F665480>, root_client=<openai.OpenAI object at 0x000002081F664AF0>, root_async_client=<openai.AsyncOpenAI object at 0x000002081F664B20>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config=

In [13]:
# retrieval chain
retriever = vectordb.as_retriever()
from langchain.chains import create_retrieval_chain

retriever_chain=create_retrieval_chain(retriever,document_chain)

In [14]:
response = retriever_chain.invoke({"input":"What is Purgatory?"})
response['answer']

'In Kafka\'s architecture, what happens if a produce or fetch request can’t complete immediately?\n\nIn Kafka\'s architecture, if a produce or fetch request can\'t complete immediately because it is waiting for some external condition (e.g., enough replicas or sufficient bytes), the operation is registered in a structure called Purgatory. This allows Kafka to avoid busy-polling or blocking threads. Instead, the delayed operations are stored and are only "woken up" when their specific conditions are met. Purgatory handles these delayed operations until they can be re-submitted to the regular request queue for a handler thread to process and send a response.'

In [15]:
response

{'input': 'What is Purgatory?',
 'context': [Document(id='6732e4bf-abfa-4c8d-af7b-ef2241b5094e', metadata={'source': 'https://medium.com/@tirthraj2004/apache-kafka-deep-dive-8f011c6483e1', 'title': 'Apache Kafka Deep Dive. Apache Kafka Internals | by Tirthraj Mahajan | Jul, 2025 | Medium', 'description': 'It prepares the environment, handles optional flags (like -daemon), and then invokes the actual Java class that runs the Kafka server Sets default JVM heap size to 1 GB if not configured. You can…', 'language': 'en'}, page_content='lets them register interest and be notified only when something changes.How It WorksEnqueueing a Delayed OperationA handler thread examines a request (produce or fetch).If its conditions aren’t met (e.g. not enough replicas, not enough bytes), it creates a DelayedOperation object and registers it with Purgatory under a key (e.g. partition ID).2. Purgatory’s Data StructuresInternally, Kafka maintains a map from keys → sets of delayed ops.There are separate p

In [16]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002081F327F70>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
            | Ch